In [27]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# try to use accelerated computing device for training
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# define the neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [28]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) # call the model this way, model.forward() will run in the bg
pred_probab = nn.Softmax(dim=1)(logits) # dim 0 is batch dim, dim 1 is prediction probabilities
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='mps:0')


## break down the layers in the FashionMNIST model
we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [37]:
# the input: tensor of 3 images
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [38]:
# the flatten layer: flattened image tensor
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [40]:
# layer1 applies a linear transformation to flat_image
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [41]:
# The non-linear transformation here is ReLU (Rectified Linear Unit).
# ReLU resets numbers < 0 to be 0, and keep number >= 0 the same.
print(f"Before ReLU: {hidden1}\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1931, -0.4149, -0.6015, -0.1277,  0.3377, -0.2601, -0.4019, -0.4808,
         -0.1818,  0.4065, -0.1671,  0.1613, -0.4092,  0.2892, -0.1395,  0.3229,
          0.0257,  0.1307,  0.2823,  0.2873],
        [ 0.1653, -0.5899, -0.8688,  0.3223,  0.2421, -0.2604, -0.6106, -0.1406,
         -0.3327, -0.1322, -0.1783,  0.1999, -0.5296,  0.2087, -0.4435, -0.0872,
          0.2054, -0.1892,  0.4954,  0.4275],
        [ 0.2393, -0.0657, -0.8361,  0.1622,  0.1395, -0.5463, -0.2437, -0.2680,
         -0.2572,  0.0530, -0.1475,  0.4746, -0.2199,  0.2692, -0.1424,  0.0947,
          0.2154, -0.5861,  0.2871,  0.3444]], grad_fn=<AddmmBackward0>)

After ReLU: tensor([[0.1931, 0.0000, 0.0000, 0.0000, 0.3377, 0.0000, 0.0000, 0.0000, 0.0000,
         0.4065, 0.0000, 0.1613, 0.0000, 0.2892, 0.0000, 0.3229, 0.0257, 0.1307,
         0.2823, 0.2873],
        [0.1653, 0.0000, 0.0000, 0.3223, 0.2421, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.1999, 0.0000, 0.2087, 0.000

In [56]:
# nn.Sequential is an ordered container of layers (modules), and data passes through all ordered layers
# nn.Sequential can be quickly used to define a network like seq_modules below

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
print(logits)

tensor([[ 0.1615,  0.0766, -0.0018, -0.2135,  0.0304,  0.0891,  0.2161,  0.2952,
          0.1532, -0.1787],
        [ 0.1674,  0.0912,  0.1385, -0.0246, -0.0701, -0.0309,  0.2904,  0.2522,
          0.2309, -0.2610],
        [ 0.1267,  0.1811,  0.1084, -0.1822, -0.0659, -0.0167,  0.2088,  0.2860,
          0.2879, -0.2167]], grad_fn=<AddmmBackward0>)


In [61]:
# the last layer of seq_modules returns logits
# logits are raw values in [-infty, infty]
# pass logits through nn.Softmax to scale the values in [0, 1] representing the model's predicted probabilities
softmax = nn.Softmax(dim=1) # dim indicates the dimension along which the values must sum to 1
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.1091, 0.1002, 0.0927, 0.0750, 0.0957, 0.1015, 0.1152, 0.1247, 0.1082,
         0.0776],
        [0.1079, 0.1000, 0.1048, 0.0890, 0.0851, 0.0885, 0.1220, 0.1174, 0.1150,
         0.0703],
        [0.1041, 0.1099, 0.1022, 0.0764, 0.0859, 0.0902, 0.1130, 0.1221, 0.1223,
         0.0738]], grad_fn=<SoftmaxBackward0>)


In [63]:
print(f"Model structure: {model}\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0032, -0.0245, -0.0245,  ..., -0.0246, -0.0323,  0.0006],
        [-0.0108,  0.0277, -0.0058,  ..., -0.0331,  0.0103,  0.0135]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0354,  0.0214], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0202,  0.0135, -0.0404,  ..., -0.0325,  0.0162, -0.0270],
        [ 0.0034, -0.0324,  0.0329,  ..., -0.0345,  0.0168, -0.0255]],
       device='mps:0', grad_fn=<SliceBack